In [1]:
import os
import sys
import glob
from os import write

from tqdm import tqdm
import pandas as pd
import numpy as np
import ast

import plotly as py
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from src.iDrink.iDrinkValPlots import ignore_id_p

sys.path.append(os.path.realpath(os.path.abspath('..')))
from iDrink import iDrinkUtilities

In [33]:
drive = iDrinkUtilities.get_drivepath()

root_iDrink = os.path.join(drive, 'iDrink')
root_val = os.path.join(root_iDrink, "validation_root")
root_stat = os.path.join(root_val, '04_Statistics')
root_omc = os.path.join(root_val, '03_data', 'OMC_new', 'S15133')
root_data = os.path.join(root_val, "03_data")
root_logs = os.path.join(root_val, "05_logs")

dir_stat_cont = os.path.join(root_stat, '01_continuous')
dir_stat_cat = os.path.join(root_stat, '02_categorical')
dir_results = os.path.join(dir_stat_cont, '01_results')
dir_plots_cont = os.path.join(dir_stat_cont, '02_plots')
dir_plots_cat = os.path.join(dir_stat_cat, '02_plots')

csv_val_trials = os.path.join(root_logs, 'validation_trials.csv')
df_val_trials = pd.read_csv(csv_val_trials, sep=';')

csv_settings = os.path.join(root_logs, 'validation_settings.csv')
df_settings = pd.read_csv(csv_settings, sep=';')

csv_calib_error = os.path.join(root_logs, 'calib_errors.csv')
df_calib_error = pd.read_csv(csv_calib_error, sep=';')

csv_murphy = os.path.join(root_stat, '02_categorical', 'murphy_measures.csv')
df_murphy = pd.read_csv(csv_murphy, sep=';')

csv_failed_trials = os.path.join(root_stat, '04_failed_trials', 'failed_trials.csv')
df_failed_trials = pd.read_csv(csv_failed_trials, sep=';')


csv_cad = os.path.join(root_stat, '02_categorical', 'clinically_acceptable_difference.csv')
df_cad = pd.read_csv(csv_cad, sep=',')

list_identifier = sorted(df_val_trials['identifier'].tolist())

ignore_id_p = ['P11', 'P19']

In [54]:
def get_title_measure_name(measure):
    """returns a string based on the murphy measure for a figure_title"""
    match measure:
        case 'PeakVelocity_mms':
            title = 'Peak Endeffector Velocity'
        case 'elbowVelocity':
            title = 'Peak Elbow Velocity'
        case 'tTopeakV_s':
            title = 'Time to Peak Velocity'
        case 'tToFirstpeakV_s':
            title = 'Time to First Peak Velocity'
        case 'tTopeakV_rel':
            title = 'Relative time to Peak Velocity relative'
        case 'tToFirstpeakV_rel':
            title = 'Relative time to First Peak Velocity'
        case 'NumberMovementUnits':
            title = 'Number of Movement Units'
        case 'InterjointCoordination':
            title = 'Interjoint Coordination'
        case 'trunkDisplacementMM':
            title = 'Trunk Displacement'
        case 'trunkDisplacementDEG':
            title = 'Trunk Displacement'
        case 'ShoulderFlexionReaching':
            title = 'Shoulder Flexion Reaching'
        case 'ElbowExtension':
            title = 'Elbow Extension'
        case 'shoulderAbduction':
            title = 'Shoulder Abduction'
        case 'shoulderFlexionDrinking':
            title = 'Shoulder Flexion Drinking'
            title = 'Trunk Angle'
        case 'hand_vel':
            title = 'Hand Velocity'
        case 'elbow_vel':
            title = 'Elbow Velocity'
        case 'trunk_disp':
            title = 'Trunk Displacement'
        case 'trunk_ang':
            title = 'Trunk Angle'
        case 'elbow_flex_pos':
            title = 'Elbow Flexion'
        case 'shoulder_flex_pos':
            title = 'Shoulder Flexion'
        case 'shoulder_abduction_pos':
            title = 'Shoulder Abduction'
        case _:
            title = ''
    return title

def get_unit(kin):

    cases_deg = ['trunk_ang', 'elbow_flex_pos', 'shoulder_flex_pos', 'shoulder_abduction_pos',
                 'trunkDisplacementDEG', 'ShoulderFlexionReaching', 'ElbowExtension',
                 'shoulderAbduction', 'shoulderFlexionDrinking']

    match kin:
        case 'hand_vel' | 'PeakVelocity_mms':
            unit = 'mm/s'
        case 'elbow_vel' | 'elbowVelocity':
            unit = 'deg/s'
        case 'trunk_disp' | 'trunkDisplacementMM':
            unit = 'mm'
        case k if k in cases_deg:
            unit = 'deg'
        case 'tTopeakV_s' | 'tToFirstpeakV_s' :
            unit = 's'
        case 'tTopeakV_rel' | 'tToFirstpeakV_rel':
            unit = '%'
        case _:
            unit = ''

    return unit

def get_cad(df, measure):
    match measure:
        case 'PeakVelocity_mms' | 'hand_vel':
            measure_name = 'peak_V'
        case 'elbowVelocity' | 'elbow_vel':
            measure_name = 'peak_V_elb'
        case 'tTopeakV_s':
            measure_name = 't_to_PV'
        case 'tToFirstpeakV_s':
            measure_name = 't_first_PV'
        case 'tTopeakV_rel':
            measure_name = 't_PV_rel'
        case 'tToFirstpeakV_rel':
            measure_name = 't_first_PV_rel'
        case 'NumberMovementUnits':
            measure_name = 'n_mov_units'
        case 'InterjointCoordination':
            measure_name = 'interj_coord'
        case 'trunkDisplacementMM' | 'trunk_disp':
            measure_name = 'trunk_disp'
        case 'trunkDisplacementDEG' | 'trunk_ang':
            return None
        case 'ShoulderFlexionReaching' | 'elbow_flex_pos':
            measure_name = 'arm_flex_reach'
        case 'ElbowExtension' | 'shoulder_flex_pos':
            measure_name = 'elb_ext'
        case 'shoulderAbduction' | 'shoulder_abduction_pos':
            measure_name = 'arm_abd'
        case 'shoulderFlexionDrinking' | 'shoulder_flex_pos':
            measure_name = 'arm_flex_drink'
        case _:
            return

    return df.loc[0, measure_name]

## kinematic measures

In [15]:
df_corr_murphy = pd.read_csv(os.path.join(dir_stat_cat, 'stat_murphy_corr.csv'), sep=';')

if 'Unnamed: 0' in df_corr_murphy.columns:
    df_corr_murphy = df_corr_murphy.drop(columns=['Unnamed: 0'])

In [20]:
len_before = len(df_corr_murphy)
df_corr_murphy.dropna(inplace=True)
len_after = len(df_corr_murphy)
print(f'Number of rows dropped: {len_before - len_after}')

Number of rows dropped: 65


In [22]:
df_grouped = df_corr_murphy.groupby(['id_s', 'measure']).mean()

C:\Users\johan\AppData\Local\Temp\ipykernel_23304\2810033707.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [24]:
df_grouped.columns

Index(['pearson', 'pearson_p'], dtype='object')

In [25]:
df_plot = pd.DataFrame(columns = ['id_s', 'measure', 'correlation', 'p_value'])

for index in df_grouped.index:
    id_s = index[0]
    measure = index[1]
    r = df_grouped.loc[index]['pearson']
    p = df_grouped.loc[index]['pearson_p']
    
    df_plot = pd.concat([df_plot, pd.DataFrame([[id_s, measure, r, p]], columns = ['id_s', 'measure', 'correlation', 'p_value'], index = [0])], ignore_index=True)

In [26]:
df_plot

,id_s,measure,correlation,p_value
0,S001,ElbowExtension,0.857928,1.426104e-61
1,S001,InterjointCoordination,0.922508,6.074450e-85
2,S001,NumberMovementUnits,0.394871,6.095553e-10
3,S001,PeakVelocity_mms,0.792647,1.785234e-38
4,S001,ShoulderFlexionReaching,0.695428,1.007662e-25
...,...,...,...,...
242,S026,tToFirstpeakV_s,0.161560,4.197175e-01
243,S026,tTopeakV_rel,-0.054279,1.998167e-01
244,S026,tTopeakV_s,0.161560,4.197175e-01
245,S026,trunkDisplacementDEG,0.238558,4.554908e-02


In [42]:
df_plot['id_s'].unique() 

array(['S001', 'S002', 'S006', 'S007', 'S008', 'S011', 'S012', 'S013',
       'S014', 'S017', 'S018', 'S019', 'S020', 'S021', 'S022', 'S023',
       'S024', 'S025', 'S026'], dtype=object)

In [43]:
id_s_multi = ['S001', 'S002', 'S006', 'S007', 'S008', 'S011', 'S012', 'S013', 'S014',]
id_s_single = ['S017', 'S018', 'S019', 'S020', 'S021', 'S022', 'S023',
       'S024', 'S025', 'S026']

In [47]:
fig = px.bar(df_plot, x='id_s', y='correlation', color='measure', title=f'Pearson Correlation', labels={'correlation': 'Pearson Correlation', 'id_s': 'Subject ID'},
             template = 'plotly', barmode="group")

fig.add_hline(y=0.9, line_dash='dot', line_color='red', name = 'minimum correlation')
fig.add_hline(y=-0.9, line_dash='dot', line_color='red', name = 'minimum correlation')

fig.update_layout(title= f'Pearson Correlation',
          xaxis_title=f'Settings',
          yaxis_title=f'Correlation [r]',
          )

fig.show()

os.makedirs(os.path.join(dir_plots_cat, '03_correlation'), exist_ok=True)

path = os.path.join(dir_plots_cat, '03_correlation', f'pearson_corr_all.html')
fig.write_html(path)

path = os.path.join(dir_plots_cat, '03_correlation', f'pearson_corr_all.png')
fig.write_image(path)

In [48]:
id_s_reduced = ['S001', 'S002', 'S008', 'S014', 'S017', 'S018', 'S019', 'S020', 'S021', 'S022', 'S023', 'S024', 'S025', 'S026']

In [52]:
fig = px.bar(df_plot[df_plot['id_s'].isin(id_s_reduced)], x='id_s', y='correlation', color='measure', title=f'Pearson Correlation', labels={'correlation': 'Pearson Correlation', 'id_s': 'Subject ID'},
             template = 'plotly', barmode="group")

fig.add_hline(y=0.9, line_dash='dot', line_color='red', name = 'minimum correlation')

fig.update_layout(title= f'Pearson Correlation',
          xaxis_title=f'Settings',
          yaxis_title=f'Correlation [r]',
          )

fig.show()

os.makedirs(os.path.join(dir_plots_cat, '03_correlation'), exist_ok=True)

path = os.path.join(dir_plots_cat, '03_correlation', f'pearson_corr_reduced.html')
fig.write_html(path)

path = os.path.join(dir_plots_cat, '03_correlation', f'pearson_corr_reduced.png')
fig.write_image(path)

In [55]:
measures = df_plot['measure'].unique()

for measure in measures:
    df_plot_measure = df_plot[df_plot['measure'] == measure]
    title_name = get_title_measure_name(measure)
    
    fig = px.bar(df_plot_measure[df_plot_measure['id_s'].isin(id_s_reduced)], x='id_s', y='correlation', title=f'Pearson Correlation of {title_name}', labels={'correlation': 'Pearson Correlation', 'id_s': 'Subject ID'},
                 template = 'plotly')
    
    fig.add_hline(y=0.9, line_dash='dot', line_color='red', name = 'minimum correlation')
    
    fig.update_layout(title= f'Pearson Correlation of {title_name}',
              xaxis_title=f'Settings',
              yaxis_title=f'Correlation [r]',
              )
    
    fig.show()
    os.makedirs(os.path.join(dir_plots_cat, '03_correlation'), exist_ok=True)
    
    path = os.path.join(dir_plots_cat, '03_correlation', f'pearson_corr_{measure}.html')
    fig.write_html(path)
    
    path = os.path.join(dir_plots_cat, '03_correlation', f'pearson_corr_{measure}.png')
    fig.write_image(path)